In [1]:
Today = '2024-04-30'
Mounth = 'Apr'

Print_ARAP = False
print_manual_map = True

import os
from datetime import date
import pandas as pd
import numpy as np

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [51]:
# query = """
# SELECT * 
# FROM "XXWH"."externalReceivableDebt"
# ORDER BY "reportDate" DESC
# FETCH FIRST 10 ROWS ONLY
# """
query = f"""
SELECT * 
FROM "XXWH"."externalReceivableDebt"
WHERE "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') 
"""

# invoiceDate
data = export_from_WHWEEK(query)
data_work = data[['companyCode', 'customerName', 'outGroupUS', 'currencyCode', 'amount', 'termDate']]
data_work.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


KeyError: "['customerName'] not in index"

In [45]:
data_work['Cty_code'] = merge_Mapping(data_work, 'customerName')
data_work['Cty_segment'] = merge_SalesUnits(data_work, 'Cty_code', merge_col='ocpSegment')
data_work['Comp_segment'] = merge_SalesUnits(data_work, 'companyCode', merge_col='ocpSegment')
data_work['Comp_holding'] = merge_SalesUnits(data_work, 'companyCode', merge_col='holding')

data_work = add_in_currency_column(data_work, 'USD', 'currencyCode', False, 'amount', Today)
data_work

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_14360\3155696699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_work['Cty_code'] = merge_Mapping(data_work, 'customerName')
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SAMSALES' 'SAMSALES' 'SAMSALES' ... 'SAMSALES' 'SAMSALES' 

,companyCode,customerName,outGroupUS,currencyCode,amount,termDate,Cty_code,Cty_segment,Comp_segment,Comp_holding,Coef_to_USD,amount_in_USD
0,BRFTO,FERTILIZANTES HERINGER SA,Internal,BRL,71893.04,2024-04-10,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,13843.154773
1,BRFTO,FERTILIZANTES HERINGER SA,Internal,BRL,89000.00,2024-04-09,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,17137.135595
2,BRFTO,MARCELINO FLORES DE OLIVEIRA,External,USD,12856.20,2024-05-15,External,External,SAMSALES,EUROCHEM,1.000000,12856.200000
3,BRFTO,FERTILIZANTES HERINGER SA,Internal,BRL,42569.50,2024-04-09,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,8196.845997
4,BRFTO,FERTILIZANTES HERINGER SA,Internal,BRL,53552.01,2024-04-09,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,10311.551200
...,...,...,...,...,...,...,...,...,...,...,...,...
28900,BRFTO,AGROVENCI COMERCIO IMPORTACAO EXPOR,External,USD,21350.00,2024-04-30,External,External,SAMSALES,EUROCHEM,1.000000,21350.000000
28901,BRFTO,GSM TRANSPORTES LTDA,External,BRL,10626.00,2023-06-08,External,External,SAMSALES,EUROCHEM,0.192552,2046.058459
28902,BRFTO,LUSENRIQUE QUINTAL,External,BRL,157760.00,2023-10-21,External,External,SAMSALES,EUROCHEM,0.192552,30377.016983
28903,BRFTO,AGROVENCI COMERCIO IMPORTACAO EXPOR,External,USD,16226.00,2024-04-30,External,External,SAMSALES,EUROCHEM,1.000000,16226.000000


In [46]:
check_data = data_work.loc[(data_work['outGroupUS'] == 'Internal') & (data_work['Cty_segment'] == 'External'),\
                                                    ['customerName', 'Cty_code', 'Cty_segment']].drop_duplicates()
check_data2 = data_work.loc[(data_work['outGroupUS'] == 'Internal') & (data_work['Comp_segment'] == 'External'),\
                                                    ['companyCode', 'Comp_segment']].drop_duplicates()

if print_manual_map == True:
    pd.concat([check_data2, check_data]).to_excel('manual_map_Braz_1.xlsx', index=False)

In [50]:
data_pivot = data_work.pivot_table(index=['Comp_holding', 'companyCode', 'Comp_segment', 'outGroupUS'], values='amount_in_USD', aggfunc={'amount_in_USD':['sum','count']}).reset_index()
data_pivot
data_pivot.to_excel(f'{str(date.today())}_externalReceivableDebt_pivot_{Mounth}.xlsx')

In [2]:
# query = """
# SELECT * 
# FROM "XXWH"."externalReceivableDebt"
# ORDER BY "reportDate" DESC
# FETCH FIRST 10 ROWS ONLY
# """
query = f"""
SELECT * 
FROM "XXWH"."externalPayableDebt"
WHERE "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD') 
"""

# invoiceDate
data = export_from_WHWEEK(query)
data_work = data[['companyCode', 'vendorName', 'outGroupUS', 'currencyCode', 'amount', 'termDate']]

data_work['Cty_code'] = merge_Mapping(data_work, 'vendorName')
data_work['Cty_segment'] = merge_SalesUnits(data_work, 'Cty_code', merge_col='ocpSegment')
data_work['Comp_segment'] = merge_SalesUnits(data_work, 'companyCode', merge_col='ocpSegment')
data_work['Comp_holding'] = merge_SalesUnits(data_work, 'companyCode', merge_col='holding')

data_work = add_in_currency_column(data_work, 'USD', 'currencyCode', False, 'amount', Today)
data_work

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9916\1916045794.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_work['Cty_code'] = merge_Mapping(data_work, 'vendorName')
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using S

,companyCode,vendorName,outGroupUS,currencyCode,amount,termDate,Cty_code,Cty_segment,Comp_segment,Comp_holding,Coef_to_USD,amount_in_USD
0,BRFTO,BELLUNO LOGISTICA E TRANSPORTES LTDA,External,BRL,5136.00,2024-04-24,External,External,SAMSALES,EUROCHEM,0.192552,9.889475e+02
1,BRFTO,FERTILIZANTES HERINGER S A,Internal,BRL,46185.30,2024-01-25,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,8.893076e+03
2,BRFTO,MOSAIC FERTILIZANTES P;K LTDA,External,BRL,53903.52,2023-07-15,External,External,SAMSALES,EUROCHEM,0.192552,1.037924e+04
3,BRFTO,FERTILIZANTES HERINGER S A,Internal,BRL,230022.18,2023-11-02,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,4.429125e+04
4,BRFTO,HIDROVIAS DO BRASIL - VILA DO CONDE S.A.,External,BRL,25082.80,2024-04-18,External,External,SAMSALES,EUROCHEM,0.192552,4.829745e+03
...,...,...,...,...,...,...,...,...,...,...,...,...
13857,BRFTO,FERTILIZANTES HERINGER S A,Internal,BRL,51317.00,2024-01-22,BRHRG,SAMSALES,SAMSALES,EUROCHEM,0.192552,9.881195e+03
13858,BRFTO,COMBITRANS AMAZONAS LTDA,External,BRL,5737.49,2024-04-08,External,External,SAMSALES,EUROCHEM,0.192552,1.104766e+03
13859,BRFTO,COMERCIAL LORENA LTDA,External,BRL,114.00,2024-04-08,External,External,SAMSALES,EUROCHEM,0.192552,2.195094e+01
13860,BRFTO,EUROCHEM TRADING GMBH,Internal,USD,2333100.00,2022-12-23,CHTRD,TRADING,SAMSALES,EUROCHEM,1.000000,2.333100e+06


In [3]:
check_data = data_work.loc[(data_work['outGroupUS'] == 'Internal') & (data_work['Cty_segment'] == 'External'),\
                                                    ['vendorName', 'Cty_code', 'Cty_segment']].drop_duplicates()
check_data2 = data_work.loc[(data_work['outGroupUS'] == 'Internal') & (data_work['Comp_segment'] == 'External'),\
                                                    ['companyCode', 'Comp_segment']].drop_duplicates()

if print_manual_map == True:
    pd.concat([check_data2, check_data]).to_excel('manual_map_Braz_1.xlsx', index=False)

In [5]:
data_pivot = data_work.pivot_table(index=['Comp_holding', 'companyCode', 'Comp_segment', 'outGroupUS'], values='amount_in_USD', aggfunc={'amount_in_USD':['sum','count']}).reset_index()
data_pivot
data_pivot.to_excel(f'{str(date.today())}_externalPayableDebt_pivot_{Mounth}.xlsx')